In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

MODEL = "gpt-3.5-turbo"
MODEL = "llama2"


In [2]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings


if MODEL.startswith ("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings()

model.invoke("Tell me a joke")

'Why was the math book sad? Because it had too many problems! 😂'

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser

chain.invoke("Tell me a joke")

"\nWhy don't scientists trust atoms? Because they make up everything! 😁"

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("mlschool.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'mlschool.pdf', 'page': 0}, page_content='Building Machine Learning Systems That Don\'t\nSuck\nA live, interactive program that\'ll help you build production-readymachine\nlearning systems from the ground up.\nNext cohort:\xa0November4 - 21, 2024\nCheck the schedulefor more details about upcoming cohorts.\nI want to join!Sign in\nLearn how to design, build, deploy, and scale machine learning\nsystems to solve real-world problems.\nI\'ll lose my mind if I see another book or course teaching people the same basic\nideas for the hundredth time. Most people are stuck in beginner mode, and finding\nhelp to solve real-world problems is hard.\nI want to change that.\nI started writing software 30 years ago. I\'ve written pipelines and trained models\nfor some of the largest companies in the world. I want to show you how to do the\nsame.\nThis is the class I wish I had taken when I started."This is the best machine learning course I\'ve done.\nWorth every cent."\n

In [4]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
print(prompt.format(context="Mary's sister is Susana", question="Who is Mary's sister?"))

Human: 
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Mary's sister is Susana

Question: Who is Mary's sister?



In [5]:
chain = prompt | model | parser


In [6]:
chain.input_schema.schema()

{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [7]:
chain.invoke({
    "context": "Mary's sister is Susana",
    "question": "Who is Mary's sister?"
})

"Of course! Based on the context provided, Mary's sister is Susana. Therefore, the answer to the question is: Susana."

In [9]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("mlschool.pdf")
pages = loader.load_and_split()
vectorstore1 = DocArrayInMemorySearch.from_documents(pages,embedding=embeddings)

/Users/derinberktay/Desktop/LLM/.venv/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [10]:
retriever = vectorstore1.as_retriever()

retriever.invoke("Machine Learning")

[Document(metadata={'source': 'mlschool.pdf', 'page': 3}, page_content="You'll learn best practices to tackle the most significant challenges machine\nlearning engineers face to build, evaluate, run, monitor, and maintain machine\nlearning systems in real-world scenarios.\nYou'll learn how to use techniques like active learning, distributed training,\nadversarial validation, human-in-the-loop deployments, model compression,\ntest-time augmentation, testing in production, among many others.\nYou'll learn how to create training, deploying, monitoring, and inference\npipelines using Amazon SageMaker and open-source tools.\nForget about theoretical concepts. This program will show you some of the things\nI've learned from real-life examples I've built during more than 30 years in the\nindustry.\nCheck the program syllabus\xa0→\nReal-life examples and case studies\nLearn from practical experience building machine\nlearning systems that work in the real world.\nLive, interactive sessions\nAs

In [11]:
from operator import itemgetter
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter ("question")
    }
    | prompt 
    | model
    | parser
)
chain.invoke ({"question": "What is machine learning?"})
    

' Based on the context provided in the question, I can answer that machine learning is a subfield of artificial intelligence that involves designing and building systems that can learn from data and make predictions or decisions without being explicitly programmed to do so. The term "machine learning" was first coined in 1956 by Arthur Samuel, and since then, it has evolved into a broad field that encompasses various techniques and approaches, such as supervised and unsupervised learning, deep learning, natural language processing, and computer vision, among others.'

In [12]:
itemgetter("question")({"question":"what is machine learning?"})

'what is machine learning?'

In [13]:
questions = [
    "What is the purpose of the course?",
    "How many hours of live sessions?",
    "How many coding assignments are there in the program?",
    "Is there a program certificate upon completion?",
    "What programming language will be used in the program?",
    "How much does the program cost?"
]
for question in questions:
    print (f"Question: {question}")
    print (f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the purpose of the course?
Answer:  Based on the context provided, the purpose of the course appears to be teaching software engineers, data scientists, data analysts, machine learning engineers, technical managers, and others how to build machine learning systems that don't suck. The course aims to provide practical skills and insights into what it takes to build systems that work in the real world. Specifically, the course will cover topics such as designing and writing code to build an end-to-end machine learning system from scratch, as well as providing feedback and support from the community.

Question: How many hours of live sessions?
Answer: The answer to the question "How many hours of live sessions?" is 4 hours per week during the three weeks of the program.

Question: How many coding assignments are there in the program?
Answer: I don't know. The document you provided does not mention the exact number of coding assignments in the program.

Question: Is there

In [16]:
for s in chain.stream({"question": "What is the purpose of the course?"}):
    print(s, end="", flush=True)

 Based on the context provided, the purpose of the course appears to be teaching students how to build machine learning systems that are effective and don't "suck." The course covers a wide range of topics, including designing and writing code for an end-to-end machine learning system, as well as providing practical skills and insights into what it takes to build systems that work in the real world. The course is designed for software engineers, data scientists, data analysts, machine learning engineers, technical managers, and anyone else who wants to use machine learning to solve real-world problems.

In [15]:
chain.batch([{"question": q} for q in questions])

[" Based on the context provided, the purpose of the course appears to be teaching students how to build machine learning systems that don't suck. The course covers a wide range of topics related to building successful machine learning systems, including designing and writing code, understanding the basics of machine learning terminology, and learning practical skills and insights into what it takes to build systems that work in the real world. The course also promises to provide lifetime access to the program with no monthly or annual fees, and offers a money-back guarantee if students are not satisfied with the course.",
 'The answer to the question is: 4 hours every week during the three weeks of the program.',
 'The document states that there are 100 coding assignments in the program.',
 'Yes, there is a program certificate upon completion of the course.',
 'I can answer the question based on the context provided. The document states that the program will use Python throughout, so 